# LDA over Time

In [5]:
from utils import * 
import plotly.plotly as py
import plotly 
import time
import datetime
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
plotly.tools.set_credentials_file(username='kt426', api_key='GcKIXRGajqJ9KlQAcL2L')

### Load dataframe

In [6]:
DATAPASS = '../../../yelp10/dataset/business.json'
business = load_json_to_df(DATAPASS)

In [7]:
DATAPASS2 = '../../../yelp10/dataset/review.json'
review = load_json_to_df(DATAPASS2)

In [8]:
categories = {}
for C in business.categories:
    for c in C:
        if c in categories:
            categories[c] += 1
        else:
            categories[c] = 1

In [9]:
categories = sorted(categories.items(), key=lambda x: x[1], reverse = True)

In [19]:
categories[:40]

[('Restaurants', 51613),
 ('Shopping', 24595),
 ('Food', 23014),
 ('Beauty & Spas', 15139),
 ('Home Services', 13202),
 ('Health & Medical', 12033),
 ('Nightlife', 11364),
 ('Bars', 9868),
 ('Automotive', 9476),
 ('Local Services', 9343),
 ('Event Planning & Services', 8038),
 ('Active Life', 7427),
 ('Fashion', 6299),
 ('Sandwiches', 5864),
 ('Fast Food', 5792),
 ('American (Traditional)', 5737),
 ('Pizza', 5652),
 ('Coffee & Tea', 5565),
 ('Hair Salons', 5395),
 ('Hotels & Travel', 5188),
 ('Arts & Entertainment', 5054),
 ('Home & Garden', 4584),
 ('Auto Repair', 4480),
 ('Italian', 4411),
 ('Burgers', 4236),
 ('Doctors', 4124),
 ('Breakfast & Brunch', 4103),
 ('Mexican', 3913),
 ('Nail Salons', 3884),
 ('Professional Services', 3865),
 ('American (New)', 3802),
 ('Chinese', 3775),
 ('Real Estate', 3729),
 ('Specialty Food', 3620),
 ('Fitness & Instruction', 3615),
 ('Pets', 3153),
 ('Grocery', 3044),
 ('Bakeries', 3014),
 ('Cafes', 2812),
 ('Hair Removal', 2704)]

In [13]:
review.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,uYHaNptLzDLoV_JZ_MuzUA,0,2016-07-12,0,VfBHSwC5Vz_pbFluy07i9Q,5,My girlfriend and I stayed here for 3 nights a...,0,cjpdDjZyprfyDG3RlkVG3w
1,uYHaNptLzDLoV_JZ_MuzUA,0,2016-10-02,0,3zRpneRKDsOPq92tq7ybAA,3,If you need an inexpensive place to stay for a...,0,bjTcT8Ty4cJZhEOEo01FGA
2,uYHaNptLzDLoV_JZ_MuzUA,0,2015-09-17,0,ne5WhI1jUFOcRn-b-gAzHA,3,Mittlerweile gibt es in Edinburgh zwei Ableger...,0,AXgRULmWcME7J6Ix3I--ww
3,uYHaNptLzDLoV_JZ_MuzUA,0,2016-08-21,0,llmdwOgDReucVoWEry61Lw,4,Location is everything and this hotel has it! ...,0,oU2SSOmsp_A8JYI7Z2JJ5w
4,uYHaNptLzDLoV_JZ_MuzUA,0,2013-11-20,0,DuffS87NaSMDmIfluvT83g,5,gute lage im stadtzentrum. shoppingmeile und s...,0,0xtbPEna2Kei11vsU-U2Mw


In [14]:
def business_id_retrieval(cat, business):
    '''
    
    Input:
        cat(str) : category
        business(dataframe) : the business data
    Output:
        id_list(set) : business ids of a particular category
    '''
    id_list = set()
    idx = 0
    for row in business.values:
        categories = row[3]
        if cat in categories:
            id_list.add(row[2])
    return id_list

## Experiment with Pizza and Chinese

In [15]:
pizza = business_id_retrieval('Pizza', business)

In [16]:
review_pizza = review.loc[review['business_id'].isin(pizza)]

In [20]:
chinese = business_id_retrieval('Chinese', business)

In [21]:
review_chinese = review.loc[review['business_id'].isin(chinese)]

### Observe review count over time

In [17]:
import plotly.graph_objs as go
data = [go.Scatter(x=review_pizza.groupby('date').count()['review_id'].keys()\
                   ,y=review_pizza.groupby('date').count()['review_id'].values)]
layout = go.Layout( title='Piazza Reviews',
    xaxis={'title':'Date'},
    yaxis={'title':'Number of reviews'},
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

In [22]:
data = [go.Scatter(x=review_chinese.groupby('date').count()['review_id'].keys()\
                   ,y=review_chinese.groupby('date').count()['review_id'].values)]
layout = go.Layout( title='Chinese Reviews',
    xaxis={'title':'Date'},
    yaxis={'title':'Number of reviews'},
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

Later, we need to normalize the growth factor

### Investigate review text, tf-idf and LDA

In [23]:
vectorizer = CountVectorizer(max_df = 0.3)
doc = vectorizer.fit_transform(review_pizza['text'])

This takes forever, don't do it on your machine

In [ ]:
freqs1 = [(word, doc.getcol(idx).sum()) for word, idx in vectorizer.vocabulary_.items()]
#sort from largest to smallest
print (sorted (freqs1, key = lambda x: -x[1])[:20])

In [24]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, max_features=1000, stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(review_pizza['text'])
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

This also takes long time, we need an access to the server

In [25]:
from sklearn.decomposition import NMF
nmf = NMF(n_components=50, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf)

In [26]:
'''
Thanks Eric Ma
'''
def display_topics(model, feature_names, n_top_words):
    for topic_index, topic in enumerate(model.components_):
        print("Topic %d:" % topic_index)
        print(" ".join([feature_names[i] for i in topic.argsort()[:-n_top_words:-1]]))

In [27]:
display_topics(nmf, tfidf_feature_names, 10)

Topic 0:
night late dinner went friday date saturday open wine
Topic 1:
pizza better hut places joint pepperoni price dough ny
Topic 2:
minutes table said wait came asked told server took
Topic 3:
order called phone online pick placed wrong customer told
Topic 4:
good pretty prices price overall bit decent value quite
Topic 5:
place eat looking clean people friends family favorite want
Topic 6:
ordered came large tasted half didn pepperoni wasn husband
Topic 7:
service customer slow fast horrible terrible bad friendly poor
Topic 8:
great atmosphere prices spot experience family fantastic service price
Topic 9:
staff friendly super helpful clean fast attentive wait family
Topic 10:
love favorite family absolutely kids coming come salads yummy
Topic 11:
fresh ingredients quality hot mozzarella tasty basil oven dough
Topic 12:
wings hot buffalo crispy ranch honey bbq fries medium
Topic 13:
und die der das ist sehr ich nicht war
Topic 14:
sauce garlic bread knots tomato red pasta marinara 

In [28]:
chinese_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, max_features=1000, stop_words='english')
chinese_tfidf = chinese_vectorizer.fit_transform(review_chinese['text'])
chinese_tfidf_feature_names = chinese_vectorizer.get_feature_names()

In [29]:
chinese_nmf = NMF(n_components=50, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(chinese_tfidf)

In [30]:
display_topics(chinese_nmf, chinese_tfidf_feature_names, 10)

Topic 0:
got table came minutes asked said didn went wait
Topic 1:
food fresh asian quality eat fast panda express court
Topic 2:
dim sum carts cart mai places har items vegas
Topic 3:
chicken orange general sweet kung tso sesame pao sour
Topic 4:
noodles hand broth dan pulled bowl noodle ramen singapore
Topic 5:
chinese authentic restaurants area american china typical real style
Topic 6:
good pretty overall prices little bit value pricey thing
Topic 7:
sushi roll fresh bar rolls sashimi tuna salmon fish
Topic 8:
egg rolls roll spring drop crab young soup puffs
Topic 9:
fried deep pan congee stir fish crab rice house
Topic 10:
dumplings dumpling steamed pan pancake lamb boiled onion shanghai
Topic 11:
delicious fresh absolutely favorite super perfect loved wonderful yummy
Topic 12:
lunch specials special dinner quick today went roll cheap
Topic 13:
rice white brown fried sticky steamed roll bowl curry
Topic 14:
soup noodle wonton bowl broth wontons base drop ton
Topic 15:
great awesom

In [34]:
print(chinese_tfidf.shape)
print (tfidf.shape)

(178151, 1000)
(268726, 1000)


## Split the review by nouns, verbs etc

In [36]:
review_pizza.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
1105,cQSQi0YWTcpuw5WGJi9XMw,0,2012-06-08,0,_iw0IGlFZtPCqxP5xcsTIQ,3,"Wie schon erwähnt, rustikales und eher altback...",0,-IbGzbx2qsGjJHm78fbj8w
1106,cQSQi0YWTcpuw5WGJi9XMw,0,2011-10-19,0,k2WNspTH6Mazkz4dxeIf4g,5,"Bodenständige italienische Küche, eher rustika...",0,bVkNsrA6W0LwEIVIKhiV-w
1107,cQSQi0YWTcpuw5WGJi9XMw,0,2011-01-10,0,BWCCewfflc2wI6AdPT8-Fg,5,Eine kleine und gemütliche Pizzeria mit lecker...,0,7EpMV3KhqozTdG2qVE1-fw
1108,cQSQi0YWTcpuw5WGJi9XMw,0,2013-01-15,0,-R-OianhKSSuLiZ6mzR19g,5,Tolle italienische Küche in einer unscheinbare...,0,4xqaWaXGO0el_jmTkm3c0A
1109,cQSQi0YWTcpuw5WGJi9XMw,2,2015-10-07,0,bokbBFumrarDfaQwMoR9Bg,5,Ein Vorurteil verwandelte sich bei dieser Pizz...,2,natSObJ4-jEev6rJRta7jA


In [119]:
def extract_word_type(s):
    tagged = nltk.pos_tag(nltk.word_tokenize(s.lower()))
    noun = [item[0] for item in tagged if item[1][0] == 'N']
    verb = [item[0] for item in tagged if item[1][0] == 'V']
    adjective = [item[0] for item in tagged if item[1][0] == 'J']
    adverb = [item[0] for item in tagged if item[1][0] == 'R']
    return ' '.join(noun), ' '.join(verb), ' '.join(adjective), ' '.join(adverb)

This takes long time, too. 

In [106]:
x = review_pizza.head(10)

In [124]:
[extract_word_type(t) for t in x['text'].values]

[('wie schon erwähnt altbackenes ist sehr freundlich wir pizza war kein zum schick essen gehen eine pizza zwischendurch eine empfehlung',
  'rustikales aber die aber',
  'eher gemütlich bedienung hatten recht passabel insgesamt laden für',
  'ambiente'),
 ('bodenständige italienische küche rustikales unedles freundliche schnarchige kellner der chef da ist es liebenswert und gemütlich immer wieder gerne hin',
  'aber',
  'sehr manchmal wenn noch nicht finde gehe',
  'ambiente'),
 ('kleine gemütliche pizzeria und gutes service',
  'mit',
  'eine leckeren pizzen preis-leistungsverhältnis freundlicher',
  'nudelgerichten'),
 ('tolle italienische küche einer location west service als speisen sind preis-leistungs verhältnis als gutschein angenommen als es ein grapa aufs haus zu empfehlen',
  'ist gut dankeschön',
  'unscheinbaren stuttgart sowohl auch die super und mehr wurde gerne gab sehr',
  ''),
 ('verwandelte bei dieser pizzeria absolute überraschung wir sind schon öfter dieser pizzeria

In [ ]:
w_type_chinese = review_chinese['text'].apply(lambda x : extract_word_type(x))

In [ ]:
w_type_pizza = review_pizza['text'].apply(lambda x : extract_word_type(x))